In [ ]:
from __future__ import print_function

import timeit
import numpy
import theano
import theano.tensor as T
import os
from rbm import RBM
from theano.tensor.shared_randomstreams import RandomStreams

from kaggle import load_data

learning_rate=0.0001
training_epochs=10
batch_size=3008
n_hidden=50

datasets = load_data()
train_set_x, train_set_y = datasets[0]
l_train_x, l_train_y = datasets[1]
l_test_x, l_test_y = datasets[2]
predict_x, predict_y = datasets[3]

# compute number of minibatches for training, validation and testing
n_train_batches = train_set_x.shape.eval()[0]// batch_size
# allocate symbolic variables for the data
index = T.lscalar()    # index to a [mini]batch
x = T.matrix('x')  # the data is presented as rasterized images

rng = numpy.random.RandomState(123)
theano_rng = RandomStreams(rng.randint(2 ** 30))

# initialize storage for the persistent chain (state = hidden
# layer of chain)
persistent_chain = theano.shared(numpy.zeros((batch_size, n_hidden),
                                             dtype=theano.config.floatX),
                                 borrow=True)

# construct the RBM class
rbm = RBM(input=x, n_visible=369,
          n_hidden=n_hidden, numpy_rng=rng, theano_rng=theano_rng)
# get the cost and the gradient corresponding to one step of CD-15
cost, updates = rbm.get_cost_updates(lr=learning_rate,
                                     persistent=persistent_chain, k=15)

#################################
#     Training the RBM          #
#################################

# start-snippet-5
# it is ok for a theano function to have no output
# the purpose of train_rbm is solely to update the RBM parameters
train_rbm = theano.function(
    [index],
    cost,
    updates=updates,
    givens={
        x: train_set_x[index * batch_size: (index + 1) * batch_size]
    },
    name='train_rbm'
)

plotting_time = 0.
start_time = timeit.default_timer()

# go through training epochs
for epoch in range(training_epochs):

    # go through the training set
    mean_cost = []
    for batch_index in range(n_train_batches):
        mean_cost += [train_rbm(batch_index)]

    print('Training epoch %d, cost is ' % epoch, numpy.mean(mean_cost))
    if abs(numpy.mean(mean_cost) ) <2 :
        break

end_time = timeit.default_timer()

pretraining_time = (end_time - start_time) - plotting_time

print ('Training took %f minutes' % (pretraining_time / 60.))

Training epoch 0, cost is  -40065.3
Training epoch 1, cost is  -58198.8
Training epoch 2, cost is  -7483.06
Training epoch 3, cost is  -796.907
Training epoch 4, cost is 

In [24]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score

l_train_=np.asarray(T.nnet.sigmoid(T.dot(l_train_x,rbm.W)).eval())
l_test_=np.asarray(T.nnet.sigmoid(T.dot(l_test_x,rbm.W)).eval())
predict_=np.asarray(T.nnet.sigmoid(T.dot(predict_x,rbm.W)).eval())

In [27]:
clf = LinearRegression()
clf.fit(l_train_,np.asarray(l_train_y.eval()))
test_y=clf.predict(l_test_)

roc_auc_score(np.asarray(l_test_y.eval()), test_y)

0.5014433626081084

In [15]:
sum(np.asarray(l_train_y.eval()))

2057.0

In [17]:
test_y

array([ 0.04178226,  0.05218485, -0.02463629, ...,  0.04178226,
        0.04178226,  0.04178226])